# Question Answering with Retrieval Augmented Generation (RAG)

## Overview

This notebook demonstrates **Question Answering (QA)** using the complete RAG pipeline. Building on `1-DocumentLoading.ipynb`, where we loaded the Digital Personal Data Protection Act (DPDPA) PDF, we now focus on the **retrieval and generation stages**:

```
Load → Split → Embed → Store → Retrieve → Generate
```

## Why RAG-based QA Matters

1. **Grounded Responses**: LLMs answer using specific documents, reducing hallucinations
2. **Source Attribution**: Metadata from document loading enables citation and audit trails
3. **Domain Specialization**: Transform generic LLMs into domain experts using your documents
4. **Up-to-date Information**: Refresh knowledge base without retraining the model

## What We'll Cover

1. **Vector Store Connection** - Loading persisted embeddings from document chunks
2. **Basic Retrieval QA** - Simple question answering with default settings
3. **Custom Prompts** - Controlling response format and preventing hallucinations
4. **Source Document Inspection** - Tracing answers back to original PDF pages
5. **Chain Type Comparison** - Evaluating `stuff`, `map_reduce`, and `refine` strategies
6. **Production Best Practices** - Guidelines for deploying RAG systems

## Architecture

```
User Question → Embed Query → Similarity Search → Retrieve Documents (with metadata) → 
LLM + Retrieved Context → Generated Answer + Citations
```

---

## Environment Setup

Initialize OpenAI client and load environment variables. These credentials are needed for:
- OpenAI embeddings (to match the embedding model used when building the vector store)
- ChatGPT models (for generating answers based on retrieved context)
- Token counting with tiktoken (for monitoring context window usage)

In [1]:
import os
import openai
import tiktoken  # OpenAI's tokenizer for accurate token counting
from dotenv import load_dotenv, find_dotenv

# Load environment variables from .env file (contains OPENAI_API_KEY)
_ = load_dotenv(find_dotenv())

openai.api_key = os.environ['OPENAI_API_KEY']

### Select LLM Model

Choose the appropriate GPT-3.5 model based on availability date. This ensures compatibility with the latest model versions while maintaining backward compatibility for older notebook executions.

In [2]:
import datetime
current_date = datetime.datetime.now().date()
if current_date < datetime.date(2023, 9, 2):
    llm_name = "gpt-3.5-turbo-0301"
else:
    llm_name = "gpt-3.5-turbo"
print(llm_name)

gpt-3.5-turbo


---

## Vector Store Connection

### Chroma Vector Database

**Chroma** is a lightweight, open-source vector database that stores:
- **Document chunks**: Text segments from split documents
- **Embeddings**: Vector representations (1536 dimensions for OpenAI)
- **Metadata**: Preserved from document loading (source, page, timestamps)

### The Journey from Document Loading to Vector Store

In `1-DocumentLoading.ipynb`, we used `PyPDFLoader` to extract the DPDPA PDF:
```python
loader = PyPDFLoader("./99-DPDPA.pdf")
pages = loader.load()  # 21 pages with metadata
```

Each page became a `Document` with rich metadata:
- `source`: `'./99-DPDPA.pdf'`
- `page`: Page index (0-20)
- `page_label`: Human-readable page number
- `total_pages`: 21
- `creator`, `producer`, `creationdate`, `moddate`: PDF properties

In subsequent notebooks, these pages were:
1. **Split** into smaller chunks for better retrieval precision
2. **Embedded** using OpenAI's text-embedding model
3. **Stored** in Chroma with metadata intact

### Loading the Persisted Collection

The next cell reconnects to the existing vector store:

In [3]:
from langchain_chroma.vectorstores import Chroma
from langchain_openai.embeddings import OpenAIEmbeddings

# Point to the directory where embeddings were persisted
persist_directory = 'docs/chroma/'

# Recreate the SAME embedding model used when building the store
# Critical: Must match the original embedding model for accurate retrieval
embedding = OpenAIEmbeddings()

# Load the existing vector store from disk (no re-embedding required)
vectordb = Chroma(
    persist_directory=persist_directory, 
    embedding_function=embedding
)

### Verify Collection Size

Check how many document chunks are stored in the vector database. The 21-page PDF was split into multiple chunks for more granular retrieval.

## Quick Retrieval Sanity Check

Before wiring an LLM into the loop, it is useful to verify that the vector store can **retrieve relevant pages** for a plain-language question.

In the next cell we:
- Define a natural-language `question` about penalties for data leaks under the DPDPA.
- Call `vectordb.similarity_search(question, k=3)` to fetch the **top 3 most similar chunks**.
- Inspect the length of `docs` to confirm we got results back.

Internally, each retrieved `Document` still carries the metadata that originated during document loading (e.g., which page of `99-DPDPA.pdf` it came from).

**Result**: 55 chunks stored in the collection.

The 21-page PDF was split into 55 smaller chunks (roughly 2-3 chunks per page on average). This chunking strategy balances:
- **Granularity**: Smaller chunks enable more precise retrieval
- **Context**: Each chunk retains enough text for coherent meaning
- **Metadata inheritance**: All chunks preserve the original page-level metadata from document loading

---

## Basic Retrieval Testing

In [4]:
print(vectordb._collection.count())

55


In [5]:
question = "What are the penalties for a data leak?"
docs = vectordb.similarity_search(question,k=3)
len(docs)

3

**Result**: Successfully retrieved 3 chunks.

The vector store found 3 document chunks related to penalties for data leaks. Each of these chunks still contains the full metadata from the original PDF loading (source file, page number, etc.), which will be crucial for providing citations when we generate answers.

### How Retrieval Works Behind the Scenes

```
Question: "What are the penalties for a data leak?"
         ↓
Query Embedding (1536-dim vector via OpenAI)
         ↓
Cosine Similarity with all 55 stored embeddings
         ↓
Top 3 most similar chunks returned
         ↓
Each chunk includes: page_content + metadata (source, page, etc.)
```

---

## Retrieval QA with LLM

Now we combine **retrieval** (finding relevant chunks) with **generation** (LLM-based answer synthesis).

### Initialize the Language Model

In [7]:
from langchain_openai import ChatOpenAI

# Initialize ChatGPT with temperature=0 for deterministic, factual responses
# Lower temperature reduces creativity/hallucination, which is ideal for QA
llm = ChatOpenAI(model=llm_name, temperature=0)

### Build the RetrievalQA Chain

`RetrievalQA` orchestrates the complete RAG pipeline:
1. Takes a user query
2. Retrieves relevant chunks from the vector store
3. Injects retrieved chunks into the LLM prompt as context
4. Returns the LLM's generated answer

**Default Behavior**: Uses the "stuff" strategy (concatenates all retrieved chunks into one prompt).

In [8]:
from langchain_classic.chains import RetrievalQA

In [9]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

### Create the QA Chain

The next cell builds a `RetrievalQA` chain by connecting:
- The ChatGPT language model
- The vector store retriever (which will fetch relevant chunks with their metadata)

## Custom QA Prompt with Retrieved Context

The basic `RetrievalQA` chain simply feeds retrieved `Document` text into the model with a default prompt. To make answers more **controlled and user-friendly**, we can define a custom prompt template.

The next cell builds a `PromptTemplate` that:
- Instructs the model to answer **only** using the provided context from the vector store (derived from `99-DPDPA.pdf`).
- Limits the answer length and enforces a fixed closing phrase ("thanks for asking!").
- Avoids hallucinations by explicitly telling the model to say "I don't know" when the context is insufficient.

This prompt design complements the **high-quality document loading and metadata** from the earlier notebook, helping the LLM stay grounded in the source PDF.

In [10]:
result = qa_chain({"query": question})

/var/folders/l8/yvtt6g7s4mg361124qs02mx40000gn/T/ipykernel_95393/4094420968.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain-classic 0.1.0 and will be removed in 1.0. Use `invoke` instead.
  result = qa_chain({"query": question})


### Run the QA Chain

Now we can ask the same question we tested earlier with retrieval-only, but this time the LLM will synthesize a natural language answer from the retrieved chunks.

In [11]:
result["result"]

'The penalties for a data leak can extend up to two hundred and fifty crore rupees, as specified in the Schedule of the Data Protection Act.'

In [12]:
from langchain_core.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

**Result**: "The penalties for a data leak can extend up to two hundred and fifty crore rupees, as specified in the Schedule of the Data Protection Act."

### Analysis

The LLM successfully:
1. **Retrieved** relevant chunks from the DPDPA (loaded via `PyPDFLoader` in earlier notebooks)
2. **Extracted** the specific penalty amount (250 crore rupees)
3. **Cited** the source ("Schedule of the Data Protection Act")
4. **Generated** a concise, factual answer

This demonstrates the complete RAG pipeline working end-to-end, from document loading to answer generation.

---

In [13]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [14]:
question = "What are all the leaks in which a penalty might be incurred to a Data Fiduciary?"

### Create QA Chain with Custom Prompt

The next cell creates a new `RetrievalQA` chain with two key enhancements:
- **return_source_documents=True**: Returns the retrieved chunks with their metadata (for citation and debugging)
- **chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}**: Injects our custom prompt template

This allows us to inspect which pages from the DPDPA PDF (loaded in `1-DocumentLoading.ipynb`) were used to answer each question.

## Comparing RetrievalQA Chain Types

`RetrievalQA.from_chain_type` supports multiple strategies for combining information from retrieved documents:

- **`stuff` (default)**: Concatenates all retrieved chunks into a single prompt (simple and fast).
- **`map_reduce`**: Runs the model separately on each chunk (map), then merges the partial answers (reduce). Helpful when documents are long or numerous.
- **`refine`**: Iteratively updates an answer as it sees more chunks, which can surface additional details.

In the following cells we reuse the **same question and vector store**, but switch the `chain_type` to `map_reduce` and `refine` to see how different strategies interpret the same DPDPA pages that were originally loaded via `PyPDFLoader`. This illustrates how downstream QA behavior depends on both the **retrieval setup** and the **document loading quality** established earlier.

### Test with a More Complex Question

Now let's ask a more nuanced question that requires synthesizing information from multiple sections of the DPDPA.

In [15]:
result = qa_chain({"query": question})

In [16]:
result["result"]

'A Data Fiduciary might incur a penalty for not protecting personal data, not reporting a personal data breach, not erasing personal data upon withdrawal of consent, not obtaining verifiable consent for processing personal data of a child or person with a disability, and not responding to grievances in a timely manner. Thanks for asking!'

**Observation**: The custom prompt is working! Notice the answer:
1. Uses concise language (3 sentences max as instructed)
2. Lists multiple penalty scenarios comprehensively
3. Ends with "Thanks for asking!" as specified in the template
4. Stays grounded in the DPDPA context

---

## Inspecting Source Documents

One of RAG's key benefits is **provenance tracking**. Because we preserved metadata during document loading (`PyPDFLoader` in `1-DocumentLoading.ipynb`), we can trace which PDF pages supported each answer.

In [17]:
result["source_documents"][0]

Document(id='d6049676-634d-41cd-965a-6a2da68e824d', metadata={'moddate': '2023-08-12T02:14:35+05:30', 'creator': 'PyPDF', 'page': 6, 'creationdate': '2023-08-12T02:13:03+05:30', 'source': './99-DPDPA.pdf', 'producer': 'iTextSharp™ 5.5.13.1 ©2000-2019 iText Group NV (AGPL-version)', 'page_label': '7', 'total_pages': 21}, page_content='to ensure effective observance of  the provisions of this Act and the rules made thereunder.\n(5) A Data Fiduciary shall protect personal data in its possession or under its control,\nincluding in respect of any processing undertaken by it or on its behalf by a Data Processor,\nby taking reasonable security safeguards to prevent personal data breach.\n(6) In the event of a personal data breach, the Data Fiduciary shall give the Board and\neach affected Data Principal, intimation of such breach in such form and manner as may be\nprescribed.\n(7) A Data Fiduciary shall, unless retention is necessary for compliance with any law\nfor the time being in force,—\

### Source Document Metadata Analysis

The retrieved document shows all the metadata preserved from `PyPDFLoader`:

**Metadata Fields**:
- `source`: `'./99-DPDPA.pdf'` - Original file from document loading
- `page`: `6` - Zero-indexed page number (7th page of PDF)
- `page_label`: `'7'` - Human-readable page number
- `total_pages`: `21` - Full document length
- `creator`, `producer`: PDF generation tools
- `creationdate`, `moddate`: Document timestamps

**Page Content**: Excerpt from Section 5-7, describing Data Fiduciary obligations including:
- Protecting personal data with reasonable security safeguards
- Reporting personal data breaches to the Board
- Erasing personal data upon consent withdrawal

This metadata enables **production features**:
1. **Citation**: "Answer based on DPDPA page 7"
2. **Audit Trail**: Track which sources influenced responses
3. **Filtering**: Retrieve only from specific documents or date ranges
4. **Version Control**: Handle multiple versions of the same document

---

In [18]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="map_reduce"
)

In [19]:
result = qa_chain_mr({"query": question})

### Map-Reduce Chain Type

The map-reduce approach processes each retrieved chunk independently, then combines the sub-answers. This can be useful for:
- Very long documents (avoiding context window limits)
- Parallel processing (faster for many chunks)
- Extracting information from each source separately

**Trade-off**: Multiple LLM calls increase cost and may miss connections between chunks.

In [20]:
result["result"]

"I don't have specific information on the types of leaks that might incur a penalty for a Data Fiduciary based on the document provided. The document focuses more on the general responsibilities and obligations of Data Fiduciaries regarding personal data processing. If you need information on common types of data breaches that could lead to penalties for Data Fiduciaries under typical data protection regulations, I can provide a general overview. Let me know if you would like me to do so."

In [21]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="refine"
)
result = qa_chain_mr({"query": question})
result["result"]

'In addition to the risk of penalties for personal data breaches, Data Fiduciaries may also face penalties for non-compliance with the provisions outlined in the Data Protection Act. This includes failing to obtain necessary information related to the personal data of Data Principals and its processing, not establishing an effective grievance redressal mechanism for Data Principals, and not responding within the specified period to grievances raised by Data Principals. Data Fiduciaries must also ensure that Data Principals can make requests for the erasure of their personal data in a specified manner. Failure to adhere to these requirements may result in penalties, fines, or legal actions as prescribed by the relevant data protection laws and regulations.'

**Observation**: The map-reduce approach produced a less specific answer!

The model admits it doesn't have "specific information" about leak types that incur penalties. This happens because:
1. Each chunk was processed **independently** (map step)
2. The reduce step couldn't fully reconstruct the comprehensive answer
3. Context connections between chunks were lost

**Lesson**: The "stuff" strategy (default) often works better for questions requiring synthesis across multiple chunks.

---

### Refine Chain Type

The refine approach iteratively improves the answer as it processes each chunk:
1. Generate initial answer from Chunk 1
2. Refine with Chunk 2
3. Further refine with Chunk 3
... and so on

This can produce more comprehensive, nuanced answers that incorporate details from all chunks.

**Observation**: The refine approach provided a more comprehensive answer!

The iterative refinement process:
1. Started with initial penalties (personal data breaches)
2. Added more details as it processed subsequent chunks
3. Produced a longer, more detailed response covering multiple non-compliance scenarios

**Comparison Summary**:

| Chain Type | Answer Quality | Cost | Speed | Use Case |
|-----------|----------------|------|-------|----------|
| **stuff** | ✅ Best for synthesis | $ | ⚡ Fast | Default choice for most questions |
| **map_reduce** | ❌ Lost context | $$$ | ⚡ Fast (parallel) | Very long documents, extractive QA |
| **refine** | ✅ Comprehensive | $$ | 🐌 Slow (sequential) | Nuanced questions, detailed analysis |

---

## Summary: Best Practices for RAG-based QA

### Production Checklist

1. **Document Loading Quality** (from `1-DocumentLoading.ipynb`)
   - Use appropriate loaders (`PyPDFLoader`, `YoutubeAudioLoader`, etc.)
   - Preserve metadata for citation and filtering
   - Validate text extraction quality

2. **Vector Store Management**
   - Use consistent embedding models across loading and querying
   - Monitor collection size and quality
   - Implement versioning for document updates

3. **Prompt Engineering**
   - Explicitly instruct models to say "I don't know" when uncertain
   - Set answer length limits to control verbosity
   - Include format requirements in the prompt

4. **Chain Type Selection**
   - Start with "stuff" (default) for most use cases
   - Use "refine" for questions requiring comprehensive answers
   - Use "map_reduce" only when context window is a constraint

5. **Source Attribution**
   - Always return source documents for citation
   - Display page numbers and source files to users
   - Log which sources influenced each answer (audit trail)

6. **Evaluation and Monitoring**
   - Test retrieval quality independently of generation
   - Monitor answer accuracy against ground truth
   - Track cost (LLM calls) and latency for each chain type

### Common Pitfalls

❌ **Mismatched embedding models**: Using different models for indexing vs. querying  
✅ **Solution**: Store embedding model version in vector store metadata

❌ **Lost metadata**: Chunking documents without preserving source information  
✅ **Solution**: Use proper document loaders that maintain metadata inheritance

❌ **Over-retrieval**: Fetching too many chunks exceeds context window  
✅ **Solution**: Tune `k` parameter, use smaller chunks, or switch to map_reduce

❌ **Hallucination**: LLM makes up answers not in retrieved context  
✅ **Solution**: Add "say I don't know" instructions to prompt, use temperature=0

### Next Steps

The complete RAG pipeline demonstrated here can be extended with:
- **Hybrid search**: Combine vector similarity with keyword matching
- **Re-ranking**: Re-score retrieved chunks for better relevance
- **Multi-query**: Generate variations of user questions for better recall
- **Conversational memory**: Add chat history for follow-up questions
- **Streaming**: Stream responses token-by-token for better UX